In [1]:
import random

In [2]:
with open('data/words_alpha.txt','r') as f:
    data =f.readlines()
    vocabulary = [x.strip('\n') for x in data]
vocabulary = set(vocabulary)

In [3]:

list(vocabulary)[:10]

['seq',
 'gastrostomize',
 'nimblewit',
 'copalite',
 'milting',
 'seam',
 'tentamen',
 'anhalouidine',
 'gemellus',
 'transelementary']

In [4]:
def process_word(word):
    word = word.lower().strip()
    word ="".join([i for i in word if i.isalpha()])
    return word

In [5]:
list_ex = ['hello','HeLlO','he56ll90o']
for i in list_ex:
    print(process_word(i))

hello
hello
hello


In [6]:
def removing_letter_ops(word):
   split_word = [(word[:i],word[i:]) for i in range(len(word)+1)]
   # print(split_word)

   delete_letter = [L + R[1:] for (L,R) in split_word if R]
   # print(delete_letter)
   return delete_letter

In [7]:
removing_letter_ops('hello')

['ello', 'hllo', 'helo', 'helo', 'hell']

In [8]:
def switch_letter_ops(word):
    split_word = [(word[:i],word[i:]) for i in range(len(word)+1)]
    # print(split_word)
    switch_letter = [L[:-1]+R[0]+L[-1]+R[1:] for L,R in split_word if (L) and (R)]
    # print(switch_letter)
    return switch_letter


In [9]:
switch_letter_ops('fast')

['afst', 'fsat', 'fats']

In [11]:
def replace_letter_ops(word):
    letters = 'abcdefghijklmnopqrstuvwxyz'
    # print(len(letters))
    split_word = [(word[:i],word[i:]) for i in range(len(word)+1)]
    # print(split_word)

    replace_letter = [L[:-1] + i + R[::] for L,R in split_word if L for i in letters]

    while word in replace_letter:
        replace_letter.remove(word)

    replace_letter = list(set(replace_letter))
    replace_letter = sorted(replace_letter)
    
    # print(len(replace_letter))
    return replace_letter

In [12]:
x = replace_letter_ops('fast')
print('first 10 words: ',x[:10])

first 10 words:  ['aast', 'bast', 'cast', 'dast', 'east', 'faat', 'fabt', 'fact', 'fadt', 'faet']


In [13]:
def insert_letter_ops(word):
    letters = 'abcdefghijklmnopqrstuvwxyz'
    split_word = [(word[:i],word[i:]) for i in range(len(word)+1)]
    # print(split_word)

    insert_letter = [L + i +R for L,R in split_word for i in letters]
    # print(len(insert_letter))
    return insert_letter

In [14]:
x = insert_letter_ops('fast')
print(x[:10])

['afast', 'bfast', 'cfast', 'dfast', 'efast', 'ffast', 'gfast', 'hfast', 'ifast', 'jfast']


In [15]:
def edit_one_letter(word):
    word = word.lower()
    one_ops = set()

    del_data = removing_letter_ops(word)
    one_ops.update(del_data)

    insert_data = insert_letter_ops(word)
    one_ops.update(insert_data)

    replace_data = replace_letter_ops(word)
    one_ops.update(replace_data)

    switch_data = switch_letter_ops(word)
    one_ops.update(switch_data)

    return set(one_ops)

In [16]:
x = edit_one_letter('cringy')
x = x & vocabulary
print(len(x))
print(list(x)[:10])


4
['clingy', 'cringe', 'fringy', 'ringy']


In [17]:
def edit_two_letter(word):
    two_ops = set()
    
    my_words = edit_one_letter(word)
    for i in my_words:
        data = edit_one_letter(i)
        two_ops.update(data)
    
    return set(two_ops)

In [18]:
x = edit_two_letter('cringy')
x = x & vocabulary
print(len(x))
print(list(x)[:10])

87
['stringy', 'craggy', 'clinty', 'rings', 'rangy', 'crony', 'swingy', 'criey', 'ching', 'bingy']


In [19]:
def edit_three_letter(word):
    three_ops = set()
    
    my_words = edit_two_letter(word)
    for i in my_words:
        data = edit_one_letter(i)
        three_ops.update(data)
    
    return set(three_ops)

In [20]:
words = ['carth']
for i in words:
    print('length of word: ', len(i))
    list1 = list(set(edit_one_letter(i)))
    # print(list1[:10])
    list2 = list(set(edit_two_letter(i)))
    # print(list2[:10])
    list3 = list(set(edit_three_letter(i)))
    print('length of first, second and third edits: ',len(list1),'    ',len(list2),'     ',len(list3))


length of word:  5
length of first, second and third edits:  285      36860       2963330


In [21]:
def suggestions_1st_time(word,vocab_list = None):
    suggestions =set()
    processed_word = process_word(word)
    if processed_word in vocab_list:
        suggestions.update({processed_word})
    
    my_words1 = edit_one_letter(word) & vocab_list
    suggestions.update(my_words1)


    words2 = edit_two_letter(word)
    my_words2 = words2 & vocab_list
    suggestions.update(my_words2)

    words2 = list(words2)
    random.seed(42)
    random.shuffle(words2)
    words2 = words2[:10000]

    words = [edit_one_letter(x) & vocab_list for x in words2] 
 
    suggestions.update(*words)
    req_words = list(my_words1) +list(my_words2-my_words1) +list(suggestions -my_words1.union(my_words2))

    return suggestions,req_words
    



In [22]:
suggestions1,words = suggestions_1st_time('abnormalization',vocab_list = vocabulary)
# print(len(suggestions1))
print(len(words))
# print(list(suggestions1)[:10])
print(words[:10])


2
['renormalization', 'normalization']


In [23]:
def suggestions_2nd_time(word2,suggestions1, vocab_list = None,n = None):
    suggestions2 = set()
    processed_word = process_word(word2)
    if processed_word in vocab_list:
        suggestions2.update({processed_word})
    
    my_words1 = edit_one_letter(word2) & vocab_list

    suggestions2.update(my_words1)

    common_suggestions = suggestions1 & suggestions2
    len_csug = len(common_suggestions)

    if len_csug>=n:
        return list(common_suggestions)
    
    words2 = edit_two_letter(word2)
    my_words2 = words2 & vocab_list 

    suggestions2.update(my_words2)

    length = len(suggestions2.union(suggestions1))

    if length >=n:
        common_suggestions = suggestions1 & suggestions2
        suggestions = list(common_suggestions) + list(suggestions2 - common_suggestions) + list(suggestions1 - common_suggestions)
        return suggestions
    

    words2 = list(words2)
    random.seed(42)
    random.shuffle(words2)

    if len(word2)>4:
        words2 = words2[:30000]

    for i in words2:
        suggestions2.update(edit_one_letter(i) & vocab_list)
        if len(suggestions2.union(suggestions1))>=n:
            break

    
    common_suggestions = suggestions1 & suggestions2
    suggestions = list(common_suggestions) + list(suggestions2 - common_suggestions) + list(suggestions1 - common_suggestions)

    return suggestions

In [24]:
word1,word2 = 'abnormalization','abnormalization'
# suggestions1,_ = correct_words_1st_time(word1,vocab_list = vocabulary)
suggestions = suggestions_2nd_time(word2,suggestions1,vocab_list=vocabulary,n = 5)
print(len(suggestions))
print(suggestions[:5])

4
['renormalization', 'normalization', 'formalization', 'normalizations']
